In [ ]:
import pandas as pd

## Задание 1
Скачайте с сайта https://grouplens.org/datasets/movielens/ датасет любого размера. Определите какому фильму было выставлено больше всего оценок 5.0.

In [ ]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')

In [ ]:
ratings.query('rating == {}'.format(5))['movieId'].value_counts().head(1)


## Задание 2
По данным файла power.csv посчитайте суммарное потребление стран Прибалтики (Латвия, Литва и Эстония) категорий 4, 12 и 21 за период с 2005 по 2010 года. Не учитывайте в расчетах отрицательные значения quantity.

In [ ]:
power = pd.read_csv('power.csv')
power.head(3)

In [ ]:
## НЕ РАБОТАЕТ
#altics = power[ power.country in ['Latvia', 'Lithuania', 'Estonia'] ]
#baltics.head()

In [ ]:
baltics = power.query('country == ["Latvia", "Lithuania", "Estonia"] and category == [4, 12, 21] and year >= 2005 and year <= 2010 and quantity > 0')
baltics.quantity.sum()

## Задание 3
Выберите страницу любого сайта с табличными данными. Импортируйте таблицы в pandas dataframe.
Примеры страниц (необязательно брать именно эти):
https://fortrader.org/quotes
https://www.finanz.ru/valyuty/v-realnom-vremeni

In [ ]:
page_url = 'https://www.finanz.ru/valyuty/v-realnom-vremeni'
course = pd.read_html(page_url, attrs = {'class': 'quote_list'}, encoding='utf-8')
course[:3]

## Задание 4
Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 4.5 и 5 - высокий рейтинг

Результат классификации запишите в столбец class

In [4]:
def classifies(chtoto):
        if chtoto <= 2:
            return 'низкий рейтинг'
        elif chtoto <= 4:
            return 'средний рейтинг'
        elif chtoto <= 4.5 or chtoto <= 5:
            return 'высокий рейтинг'

In [5]:
classifiedRating = ratings.merge(movies, on=["movieId"], how='inner').pivot_table(index=['title','movieId'], values = 'rating', aggfunc='mean')

In [7]:
classifiedRating['class'] = classifiedRating['rating'].apply(classifies)
classifiedRating.head()

,,rating,class
title,movieId,,
"""BLOW THE NIGHT!"" Let's Spend the Night Together (1983)",208297,3.000000,средний рейтинг
"""Great Performances"" Cats (1998)",51372,2.896648,средний рейтинг
#1 Cheerleader Camp (2010),136604,2.111111,средний рейтинг
#Captured (2017),183901,3.750000,средний рейтинг
#Female Pleasure (2018),195955,3.666667,средний рейтинг


## Задание 5
Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

Результат классификации запишите в отдельный столбец region.

In [8]:
keywords = pd.read_csv('keywords.csv')
keywords.head()

geo_data = {
    'Центр': ['москва', 'тула', 'ярославль'],
    'Северо-Запад': ['петербург', 'псков', 'мурманск'],
    'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}

In [9]:
temp = {}

for region in geo_data:
    cityRegion = geo_data[region]
    temp.update({key:value for key,value in zip(cityRegion, [region for r in cityRegion])})

keywords['region'] = keywords['keyword'].map(temp)
keywords['region'] = keywords['region'].fillna('undefined')
keywords.head()

,keyword,shows,region
0,вк,64292779,undefined
1,одноклассники,63810309,undefined
2,порно,41747114,undefined
3,ютуб,39995567,undefined
4,вконтакте,21014195,undefined


## Задание 6
Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

В переменную years запишите список из всех годов с 1950 по 2010.

Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

для каждой строки пройдите по всем годам списка years
если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год
Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [10]:
total = ratings.merge(movies, on=["movieId"], how='inner')
total.head()

,userId,movieId,rating,timestamp,title,genres
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller


In [14]:
def production_year(title):
    for year in years:
        if str(year) in title:
            return year
    return 1900

In [12]:
years = []
for year in range(1950,2010):
    years.append(year)

In [16]:
total['year'] = total['title'].apply(production_year)
total.head()

,userId,movieId,rating,timestamp,title,genres,year
0,1,296,5.0,1147880044,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
1,3,296,5.0,1439474476,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
2,4,296,4.0,1573938898,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
3,5,296,4.0,830786155,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994
4,7,296,4.0,835444730,Pulp Fiction (1994),Comedy|Crime|Drama|Thriller,1994


In [17]:
totalBig = total.pivot_table(index=['year'], values = 'rating', aggfunc='mean')
totalSort = totalBig.sort_values(by = ['rating'], ascending = False)
totalSort.head()

,rating
year,
1957,3.999492
1954,3.985693
1972,3.956477
1962,3.929445
1952,3.895315
